# Imports

In [1]:
import numpy as np
import pandas as pd
import math as math
import datetime as dt
import fitz as fitz
from enum import Enum

# Classes and Enums

In [2]:
# using as a value tracker

class RoomOrder(Enum):
	ABF = 0 # 18a, 18b, 18
	BAF = 1 # 18b, 18a, 18
	AFB = 2 # 18a, 18, 18b
	BFA = 3 # 18b, 18, 18a
	FAB = 4 # 18, 18a, 18b
	FBA = 5 # 18, 18b, 18a
	BA = 6 # 18b, 18a
	BF = 7 # 18b, 18
	FB = 8 # 18, 18b
	FA = 9 # 18, 18a
	AB = 10 # 18a, 18b
	AF = 11 # 18a, 18
	A = 12 # 18a
	B = 13 # 18b
	F = 14 # 18

class GroupComp(Enum):
	INDIVIDUAL = 0
	GROUP = 1

class LecternsViewed(Enum):
	N = 0 # None
	W = 1 # West
	M = 2 # Middle
	E = 3 # East
	WM = 4 # West, Middle
	WE = 5 # West, East
	ME = 6 # Middle, East
	WME = 7 # West, Middle, East

class VisitorType(Enum):
	BROWSER = 0
	FOLLOWER = 1
	SEARCHER = 2
	RESEARCHER = 3

class TurnDirection(Enum):
	LEFT = 0
	MIDDLE = 1
	RIGHT = 2
	NONE = 3

class TeamMember(Enum):
	Courtney = 0
	Jerry = 1
	Owen = 2
	Ritvik = 3
	Sofia = 4

class Gender(Enum):
	FEMALE = 0
	MALE = 1

class FirstTurnDirection(Enum):
	LEFT = 0
	MIDDLE = 1
	RIGHT = 2
	NONE = 3

class DayOfWeek(Enum):
	Sunday = 0
	Monday = 1
	Tuesday = 2
	Wednesday = 3
	Thursday = 4
	Friday = 5
	Saturday = 6

In [3]:
def raise_input_exception(visitor_id: str, cell_loc: str) -> None:
	raise Exception("Incorrect code in sheet " + visitor_id + " cell " + cell_loc + ".")

def get_team_member(visitor_id: str, cell_val: str, cell_loc: str) -> str:
	match cell_val.lower():
		case "c":
			return "Courtney"
		case "j":
			return "Jerry"
		case "o":
			return "Owen"
		case "r":
			return "Ritvik"
		case "s":
			return "Sofia"
		case _:
			raise_input_exception(visitor_id, cell_loc)

def get_group_comp(visitor_id: str, cell_val: str, cell_loc: str) -> str:
	match cell_val.lower():
		case "i":
			return "INDIVIDUAL"
		case "g":
			return "GROUP"
		case _:
			raise_input_exception(visitor_id, cell_loc)

def get_gender(visitor_id: str, cell_val: str, cell_loc: str) -> str:
	match cell_val.lower():
		case "f":
			return "FEMALE"
		case "m":
			return "MALE"
		case _:
			raise_input_exception(visitor_id, cell_loc)

def get_room_order(visitor_id: str, first_row: int, col: int, cell_loc: str, xlsx_sheet: pd.DataFrame) -> str:
	first_cell = str(xlsx_sheet.loc[first_row, col])
	second_cell = str(xlsx_sheet.loc[first_row + 1, col])
	third_cell = str(xlsx_sheet.loc[first_row + 2, col])
	first_cell_nan = xlsx_sheet.isnull().loc[first_row, col]
	second_cell_nan = xlsx_sheet.isnull().loc[first_row + 1, col]
	third_cell_nan = xlsx_sheet.isnull().loc[first_row + 2, col]
	
	if first_cell == "18":
		if second_cell == "18a":
			if third_cell == "18b":
				return "FAB"
			elif third_cell_nan:
				return "FA"
			else:
				raise_input_exception(visitor_id, cell_loc)
		elif second_cell == "18b":
			if third_cell == "18a":
				return "FBA"
			elif third_cell_nan:
				return "FB"
			else:
				raise_input_exception(visitor_id, cell_loc)
		elif second_cell_nan:
			if third_cell_nan:
				return "F"
			else:
				raise_input_exception(visitor_id, cell_loc)
		else:
			raise_input_exception(visitor_id, cell_loc)
	elif first_cell == "18a":
		if second_cell == "18":
			if third_cell == "18b":
				return "AFB"
			elif third_cell_nan:
				return "AF"
			else:
				raise_input_exception(visitor_id, cell_loc)
		elif second_cell == "18b":
			if third_cell == "18":
				return "ABF"
			elif third_cell_nan:
				return "AB"
			else:
				raise_input_exception(visitor_id, cell_loc)
		elif second_cell_nan:
			if third_cell_nan:
				return "A"
			else:
				raise_input_exception(visitor_id, cell_loc)
		else:
			raise_input_exception(visitor_id, cell_loc)
	elif first_cell == "18b":
		if second_cell == "18":
			if third_cell == "18a":
				return "BFA"
			elif third_cell_nan:
				return "BF"
			else:
				raise_input_exception(visitor_id, cell_loc)
		elif second_cell == "18a":
			if third_cell == "18":
				return "BAF"
			elif third_cell_nan:
				return "BA"
			else:
				raise_input_exception(visitor_id, cell_loc)
		elif second_cell_nan:
			if third_cell_nan:
				return "B"
			else:
				raise_input_exception(visitor_id, cell_loc)
		else:
			raise_input_exception(visitor_id, cell_loc)
	else:
		raise_input_exception(visitor_id, cell_loc)

def get_lecterns_viewed(visitor_id: str, first_row: int, col: int, cell_loc: str, xlsx_sheet: pd.DataFrame) -> str:
	first_cell = xlsx_sheet.loc[first_row, col]
	second_cell = xlsx_sheet.loc[first_row + 1, col]
	third_cell = xlsx_sheet.loc[first_row + 2, col]
	first_cell_nan = xlsx_sheet.isnull().loc[first_row, col]
	second_cell_nan = xlsx_sheet.isnull().loc[first_row + 1, col]
	third_cell_nan = xlsx_sheet.isnull().loc[first_row + 2, col]
	
	if not ((first_cell == "West" or first_cell_nan) and (second_cell == "Middle" or second_cell_nan) and (third_cell == "East" or third_cell_nan)):
		raise_input_exception(visitor_id, cell_loc)
	
	if not first_cell_nan:
		if not second_cell_nan:
			if not third_cell_nan:
				return "WME"
			else:
				return "WM"
		elif not third_cell_nan:
			return "WE"
		else:
			return "W"
	elif not second_cell_nan:
		if not third_cell_nan:
			return "ME"
		else:
			return "M"
	elif not third_cell_nan:
		return "E"
	else:
		return "N"

def get_day_of_week(visitor_id: str, cell_val: str, cell_loc: str) -> str:
	match cell_val.lower():
		case "su":
			return "Sunday"
		case "m":
			return "Monday"
		case "t":
			return "Tuesday"
		case "w":
			return "Wednesday"
		case "r":
			return "Thursday"
		case "f":
			return "Friday"
		case "sa":
			return "Saturday"
		case _:
			raise_input_exception(visitor_id, cell_loc)

def get_first_turn(visitor_id: str, cell_val: str, cell_loc: str) -> str:
	match cell_val.lower():
		case "l":
			return "LEFT"
		case "m":
			return "MIDDLE"
		case "r":
			return "RIGHT"
		case "n":
			return "NONE"
		case _:
			raise_input_exception(visitor_id, cell_loc)
def get_visitor_type(visitor_id: str, cell_val: str, cell_loc: str) -> str:
	match cell_val.lower():
		case "b":
			return "BROWSER"
		case "f":
			return "FOLLOWER"
		case "s":
			return "SEARCHER"
		case "r":
			return "RESEARCHER"
		case _:
			raise_input_exception(visitor_id, cell_loc)

# Code

## Constants

In [4]:
# TODO: update all values accordingly
# number of observations made by each pair
num_observations_0XX: int = 47
num_observations_1XX: int = 53

# total number of observations
num_observations: int = num_observations_0XX + num_observations_1XX

# name of the master tracking sheet file
master_tracker_name: str = "Master Visitor Tracker Sheet"

# relative file paths
xlsx_path: str = "../assets/excel_files/Direct_Observation_Data.xlsx"
tracker_folder_path: str = "../assets/tracker_images"
export_path: str = "../assets/excel_files/observation_tables.xlsx"

# export file sheet names
main_sheet_name: str = "main data"
groups_sheet_name_base: str = "object group"
indiv_sheet_name_base: str = "object indiv"

## Initializing Tables and Lists

In [5]:
# column names

## SURVEY GRAPHING: 1-5 is bw, multiple choice is pie, select all is stacked bar

DATA_COLUMNS: dict = {
	"visitor_id": str,
	"observer": str,
	"tracker": str,
	"time_in": str, # datetime
	"time_out": str, # datetime
	"total_time": str, # timedelta # bw
	"group_comp": str, # pie
	"gender": str, # pie
	"room_order": str, # pie
	"lecterns_visited": str, # pie
	"visitor_type": str, # pie
	"chatted_with_visitors": bool, # COMBINE ALL NOTABLE BEHAVIORS AS % BAR CHART
	"chatted_with_staff": bool, # COMBINE ALL NOTABLE BEHAVIORS AS % BAR CHART
	"sat_on_bench": bool,  # COMBINE ALL NOTABLE BEHAVIORS AS % BAR CHART
	"split_from_group": bool, # COMBINE ALL NOTABLE BEHAVIORS AS % BAR CHART
	"used_phone": bool, # COMBINE ALL NOTABLE BEHAVIORS AS % BAR CHART
	"used_museum_guide": bool, # COMBINE ALL NOTABLE BEHAVIORS AS % BAR CHART
	"used_headphones": bool, # COMBINE ALL NOTABLE BEHAVIORS AS % BAR CHART
	"first_turn_direction": str, # pie
	"18a_entrance_time": str, # datetime
	"18a_exit_time": str, # datetime
	"18a_total_time": str, # timedelta, populate post-fill # bw
	"18a_took_photos": bool, # statistics
	"18a_took_videos": bool, # statistics
	"18a_viewed_labels": bool, # statistics
	"18b_entrance_time": str, # datetime
	"18b_exit_time": str, # datetime
	"18b_total_time": str, # timedelta, populate post-fill # bw
	"18b_touched_casts": bool, # statistics
	"18b_took_photos": bool, # statistics
	"18b_viewed_labels": bool, # statistics
	"tracker_map_path": str, # NA
	"additional_notes": str # NA
}
OBJECT_GROUPS_COLUMNS: dict = {
	"group_name": str,
	"visit_order": int,
	"dwell_time": str, # timedelta # bw
	"revisited": bool, # only for friezes
	"viewed_labels": bool, # only for friezes
	"took_photos": bool
}
OBJECT_GROUP_NAMES: list = [
	"South Metopes II-V",
	"West Pediments",
	"South Metopes VI-IX",
	"South Metopes XXVI-XXIX",
	"East Pediments",
	"South Metopes XXX-XXXII",
	"Frieze Section 1",
	"Frieze Section 2",
	"Frieze Section 3",
	"Frieze Section 4", 
	"Frieze Section 5"
]
OBJECT_INDIV_COLUMNS: dict = {
	"object_name": str,
	"revisited": bool, # mutually exclusive with 'visited'; applies to metopes/pediments
	"visited": bool, # mutually exclusive with 'revisited'; applies to friezes
	"viewed_labels": bool,
	"took_photos": bool
}
OBJECT_NAMES: list = [
	"South Metope II",
	"South Metope III",
	"South Metope IV",
	"West Pediment A*",
	"West Pediment A (back)*",
	"West Pediment H",
	"West Pediment L*",
	"West Pediment M",
	"West Pediment N*",
	"West Pediment N (back)",
	"West Pediment O",
	"West Pediment Q",
	"South Metope V",
	"South Metope VI",
	"South Metope VII",
	"South Metope VIII",
	"South Metope IX",
	"East Pediment A-C*",
	"East Pediment D*",
	"East Pediment D (back)",
	"East Pediment E and F",
	"East Pediment E,F (back)",
	"East Pediment G*",
	"East Pediment G (back)",
	"East Pediment K",
	"East Pediment L and M*",
	"East Pediment K,L,M (back)",
	"East Pediment O",
	"South Metope XXVI",
	"South Metope XXVII*",
	"South Metope XXVIII",
	"South Metope XXIX",
	"South Metope XXX",
	"South Metope XXXI",
	"South Metope XXXII",
	"South Frieze XXXI 78-79*",
	"East Frieze III 7-11*",
	"East Frieze IV 24-25*",
	"East Frieze III 3-35*",
	"North Frieze V 13*",
	"North Frieze XXVII 73-74*",
	"North Frieze XLVII 132-136*",
	"West Frieze 2-3*",
]

## Fetching Data

### Loading XLSX

In [6]:
xlsx: pd.ExcelFile = pd.ExcelFile(xlsx_path)
xlsx_dfs: list = list()

for i in range(1, num_observations_0XX + 1):
	sheet_name: str = "0" + "{0:0=2d}".format(i)
	new_sheet: pd.DataFrame = pd.read_excel(xlsx, sheet_name=sheet_name, header=None)
	new_sheet.replace('nan', np.nan)
	xlsx_dfs.append(new_sheet)
for i in range(1, num_observations_1XX + 1):
	sheet_name: str = "1" + "{0:0=2d}".format(i)
	new_sheet: pd.DataFrame = pd.read_excel(xlsx, sheet_name=sheet_name, header=None)
	new_sheet.replace('nan', np.nan)
	xlsx_dfs.append(new_sheet)	

### Formatting PDFs

In [7]:
skipped_msg: str = ""

def remove_other_page(file_name: str) -> str:
	path: str = tracker_folder_path + "/" + file_name
	file: fitz.Document = fitz.open(path)
	if file.page_count == 2:
		file.delete_page(1)
		file.saveIncr()
		return ""
	elif file.page_count == 1:
		return file_name + ", "
	else:
		raise Exception("Incompatible page count: 0" + "{0:0=2d}".format(i) + ".pdf")

def remove_master_page(file_name: str) -> str:
	path: str = tracker_folder_path + "/" + file_name
	file: fitz.Document = fitz.open(path)
	if file.page_count == 2:
		file.delete_page(1)
		page: fitz.Page = file[0]
		media_box: fitz.Rect = page.mediabox
		new_rect: fitz.Rect = fitz.Rect(
			media_box.x0 + 75,
			media_box.y0,
			media_box.x1,
			media_box.y1
		)

		page.set_cropbox(new_rect)

		file.saveIncr()
		return ""
	elif file.page_count == 1:
		return file_name + ", "
	else:
		raise Exception("Incompatible page count: 0" + "{0:0=2d}".format(i) + ".pdf")

skipped_msg += remove_master_page(master_tracker_name + ".pdf")

for i in range(1, num_observations_0XX + 1):
	file_name: str = "0" + "{0:0=2d}".format(i) + ".pdf"
	skipped_msg += remove_other_page(file_name)
	
for i in range(1, num_observations_1XX + 1):
	file_name: str = "1" + "{0:0=2d}".format(i) + ".pdf"
	skipped_msg += remove_other_page(file_name)

skipped_msg = skipped_msg[0:-2]
print("SKIPPED: " + skipped_msg)

SKIPPED: Master Visitor Tracker Sheet.pdf, 001.pdf, 002.pdf, 003.pdf, 004.pdf, 005.pdf, 006.pdf, 007.pdf, 008.pdf, 009.pdf, 010.pdf, 011.pdf, 012.pdf, 013.pdf, 014.pdf, 015.pdf, 016.pdf, 017.pdf, 018.pdf, 019.pdf, 020.pdf, 021.pdf, 022.pdf, 023.pdf, 024.pdf, 025.pdf, 026.pdf, 027.pdf, 028.pdf, 029.pdf, 030.pdf, 031.pdf, 032.pdf, 033.pdf, 034.pdf, 035.pdf, 036.pdf, 037.pdf, 038.pdf, 039.pdf, 040.pdf, 041.pdf, 042.pdf, 043.pdf, 044.pdf, 045.pdf, 046.pdf, 047.pdf, 101.pdf, 102.pdf, 103.pdf, 104.pdf, 105.pdf, 106.pdf, 107.pdf, 108.pdf, 109.pdf, 110.pdf, 111.pdf, 112.pdf, 113.pdf, 114.pdf, 115.pdf, 116.pdf, 117.pdf, 118.pdf, 119.pdf, 120.pdf, 121.pdf, 122.pdf, 123.pdf, 124.pdf, 125.pdf, 126.pdf, 127.pdf, 128.pdf, 129.pdf, 130.pdf, 131.pdf, 132.pdf, 133.pdf, 134.pdf, 135.pdf, 136.pdf, 137.pdf, 138.pdf, 139.pdf, 140.pdf, 141.pdf, 142.pdf, 143.pdf, 144.pdf, 145.pdf, 146.pdf, 147.pdf, 148.pdf, 149.pdf, 150.pdf, 151.pdf, 152.pdf, 153.pdf


## Transferring Data

In [8]:
# main database
visitor_df: pd.DataFrame = pd.DataFrame({c: pd.Series(dtype=t) for c, t in DATA_COLUMNS.items()})

# lists of sub-databases
object_groups_dfs: dict = dict()
object_indiv_dfs: dict = dict()

for i in range(1, num_observations + 1):
	xlsx_sheet: pd.DataFrame = xlsx_dfs[i-1]

	# enum vars
	visitor_id: str = "{0:0=3d}".format(xlsx_sheet.loc[4, 0])

	observer: str = get_team_member(visitor_id, xlsx_sheet.loc[1, 0].strip(), "A2")
	tracker: str = get_team_member(visitor_id, xlsx_sheet.loc[1, 1].strip(), "B2")
	
	group_comp: str = get_group_comp(visitor_id, xlsx_sheet.loc[4, 1], "B5")
	
	gender: str = get_gender(visitor_id, xlsx_sheet.loc[4, 2], "C5")
	
	room_order: str = get_room_order(visitor_id, 4, 4, "E5-E7", xlsx_sheet)
	
	lecterns_viewed: str = get_lecterns_viewed(visitor_id, 4, 5, "F5-F7", xlsx_sheet)
	
	day_of_week: str = get_day_of_week(visitor_id, xlsx_sheet.loc[1, 5], "F2")

	first_turn: str = get_first_turn(visitor_id, xlsx_sheet.loc[4, 3], "D5")
	
	visitor_type: str = get_visitor_type(visitor_id, xlsx_sheet.loc[6, 3], "D7")

	# create main data
	new_visitor_data: dict = {
		"visitor_id": visitor_id,
		"observer": observer,
		"tracker": tracker,
		"time_in": 0 if xlsx_sheet.isnull().loc[1, 2] else xlsx_sheet.loc[1, 2],
		"time_out": 0 if xlsx_sheet.isnull().loc[1, 3] else xlsx_sheet.loc[1, 3],
		"total_time": 0 if xlsx_sheet.isnull().loc[1, 4] else xlsx_sheet.loc[1, 4],
		"group_comp": group_comp,
		"gender": gender,
		"room_order": room_order,
		"lecterns_visited": lecterns_viewed,
		"visitor_type": visitor_type,
		"day_of_week": day_of_week,
		"chatted_with_visitors": eval(str(xlsx_sheet.loc[1, 8])),
		"chatted_with_staff": eval(str(xlsx_sheet.loc[2, 8])),
		"sat_on_bench": eval(str(xlsx_sheet.loc[3, 8])),
		"split_from_group": eval(str(xlsx_sheet.loc[4, 8])),
		"used_phone": eval(str(xlsx_sheet.loc[5, 8])),
		"used_museum_guide": eval(str(xlsx_sheet.loc[6, 8])),
		"used_headphones": eval(str(xlsx_sheet.loc[7, 8])),
		"first_turn_direction": first_turn,
		"18a_entrance_time": 0 if xlsx_sheet.isnull().loc[27, 5] else xlsx_sheet.loc[27, 5],
		"18a_exit_time": 0 if xlsx_sheet.isnull().loc[27, 6] else xlsx_sheet.loc[27, 6],
		"18a_total_time": 0,
		"18a_took_photos": eval(str(xlsx_sheet.loc[27, 7])),
		"18a_took_videos": eval(str(xlsx_sheet.loc[27, 8])),
		"18a_viewed_labels": eval(str(xlsx_sheet.loc[27, 9])),
		"18b_entrance_time": 0 if xlsx_sheet.isnull().loc[31, 5] else xlsx_sheet.loc[31, 5],
		"18b_exit_time": 0 if xlsx_sheet.isnull().loc[31, 6] else xlsx_sheet.loc[31, 6],
		"18b_total_time": 0,
		"18b_touched_casts": eval(str(xlsx_sheet.loc[31, 7])),
		"18b_took_photos": eval(str(xlsx_sheet.loc[31, 8])),
		"18b_viewed_labels": eval(str(xlsx_sheet.loc[31, 9])),
		"tracker_map_path": tracker_folder_path + "/" + visitor_id + ".pdf",
		"additional_notes": xlsx_sheet.loc[18, 1]
	}

	new_visitor_groups_data: pd.DataFrame = pd.DataFrame({c: pd.Series(dtype=t) for c, t in OBJECT_GROUPS_COLUMNS.items()})
	for row_name in OBJECT_GROUP_NAMES:
		new_object_group_data: dict

		if "Frieze" in row_name:
			row = xlsx_sheet.loc[xlsx_sheet[4].str.strip() == row_name].squeeze()
			if row.empty:
				raise Exception("Error in row fetching: " + row_name + ", sheet: " + visitor_id)

			new_object_group_data = {
				"group_name": row_name,
				"visit_order": row[5],
				"dwell_time": 0 if row.isna()[6] else row[6],
				"revisited": str(row[7]) == "True",
				"viewed_labels": str(row[8]) == "True",
				"took_photos": str(row[9]) == "True"
			}
		else:
			cell = xlsx_sheet.loc[11, 5]
			row = xlsx_sheet.loc[xlsx_sheet[5].str.strip() == row_name].squeeze()
			if row.empty:
				raise Exception("Error in row fetching: " + row_name + ", sheet: " + visitor_id)

			new_object_group_data = {
				"group_name": row_name,
				"visit_order": row[6],
				"dwell_time": 0 if row.isna()[7] else row[7],
				"took_photos": str(row[8]) == "True"
			}

		new_visitor_groups_data.loc[new_visitor_groups_data.shape[0]] = new_object_group_data

	new_visitor_indiv_data: pd.DataFrame = pd.DataFrame({c: pd.Series(dtype=t) for c, t in OBJECT_INDIV_COLUMNS.items()})
	for row_name in OBJECT_NAMES: # add row correspondingly
		new_object_data: dict

		if "Frieze" in row_name:
			row = xlsx_sheet.loc[xlsx_sheet[0].str.strip() == row_name].squeeze()
			if row.empty:
				raise Exception("Error in row fetching: " + row_name + ", sheet: " + visitor_id)
		
			new_object_data = {
				"object_name": row_name,
				"revisited": np.nan,
				"visited": str(row[3]) == "True",
				"viewed_labels": str(row[1]) == "True",
				"took_photos": str(row[2]) == "True"
			}
		else:
			row = xlsx_sheet.loc[xlsx_sheet[11].str.strip() == row_name].squeeze()
			if row.empty:
				raise Exception("Error in row fetching: " + row_name + ", sheet: " + visitor_id)
			
			new_object_data = {
				"object_name": row_name,
				"revisited": str(row[14]) == "True",
				"visited": np.nan,
				"viewed_labels": str(row[12]) == "True",
				"took_photos": str(row[13]) == "True"
			}
		
		new_visitor_indiv_data.loc[new_visitor_indiv_data.shape[0]] = new_object_data

	# add data to tables and lists
	visitor_df.loc[i-1] = new_visitor_data
	object_groups_dfs[new_visitor_data["visitor_id"]] = new_visitor_groups_data
	object_indiv_dfs[new_visitor_data["visitor_id"]] = new_visitor_indiv_data

## Formatting Rows

In [ ]:
visitor_df = visitor_df.astype({
	"observer": str,
	"tracker": str,
	"group_comp": str,
	"gender": str, 
	"room_order": str,
	"lecterns_visited": str,
	"visitor_type": str,
	"chatted_with_visitors": "Int64",
	"chatted_with_staff": "Int64",
	"sat_on_bench": "Int64",
	"split_from_group": "Int64",
	"used_phone": "Int64",
	"used_museum_guide": "Int64",
	"used_headphones": "Int64",
	"first_turn_direction": str,
	"18a_took_photos": "Int64",
	"18a_took_videos": "Int64",
	"18a_viewed_labels": "Int64",
	"18b_touched_casts": "Int64",
	"18b_took_photos": "Int64",
	"18b_viewed_labels": "Int64",
	"tracker_map_path": str,
	"additional_notes": str
})

visitor_df = visitor_df.astype({
	"chatted_with_visitors": "boolean",
	"chatted_with_staff": "boolean",
	"sat_on_bench": "boolean",
	"split_from_group": "boolean",
	"used_phone": "boolean",
	"used_museum_guide": "boolean",
	"used_headphones": "boolean",
	"18a_took_photos": "boolean",
	"18a_took_videos": "boolean",
	"18a_viewed_labels": "boolean",
	"18b_touched_casts": "boolean",
	"18b_took_photos": "boolean",
	"18b_viewed_labels": "boolean"
})

visitor_df["time_in"] = pd.to_datetime(visitor_df["time_in"], format="%H:%M:%S")
visitor_df["time_out"] = pd.to_datetime(visitor_df["time_out"], format="%H:%M:%S")
visitor_df["18a_entrance_time"] = pd.to_datetime(visitor_df["18a_entrance_time"], format="%H:%M:%S", errors="coerce")
visitor_df["18a_exit_time"] = pd.to_datetime(visitor_df["18a_exit_time"], format="%H:%M:%S", errors="coerce")
visitor_df["18b_entrance_time"] = pd.to_datetime(visitor_df["18b_entrance_time"], format="%H:%M:%S", errors="coerce")
visitor_df["18b_exit_time"] = pd.to_datetime(visitor_df["18b_exit_time"], format="%H:%M:%S", errors="coerce")

visitor_df["18a_total_time"] = (visitor_df["18a_exit_time"] - visitor_df["18a_entrance_time"]).dt.total_seconds()
visitor_df["18b_total_time"] = (visitor_df["18b_exit_time"] - visitor_df["18b_entrance_time"]).dt.total_seconds()

for v_id, v_df in object_groups_dfs.items():
	v_df = v_df.astype({
		"group_name": str,
		"visit_order": "Int64", # actually int, rest are bools
		"revisited": "Int64",
		"viewed_labels": "Int64",
		"took_photos": "Int64"
	})
	v_df = v_df.astype({
		"revisited": "boolean",
		"viewed_labels": "boolean",
		"took_photos": "boolean"
	})
	
	object_groups_dfs[v_id] = v_df

for v_id, v_df in object_indiv_dfs.items():
	v_df = v_df.astype({
		"object_name": str,
		"revisited": "Int64",
		"visited": "Int64",
		"viewed_labels": "Int64",
		"took_photos": "Int64"
	})
	v_df = v_df.astype({
		"revisited": "boolean",
		"visited": "boolean",
		"viewed_labels": "boolean",
		"took_photos": "boolean"
	})
	
	object_indiv_dfs[v_id] = v_df

## Exporting Databases

In [ ]:
with pd.ExcelWriter(export_path) as write_file:
	visitor_df.to_excel(write_file, sheet_name=main_sheet_name)
	for v_id in object_groups_dfs.keys():
		groups_sheet_name = groups_sheet_name_base + " " + v_id
		object_groups_dfs[v_id].to_excel(write_file, sheet_name=groups_sheet_name, index=False)
		indiv_sheet_name = indiv_sheet_name_base + " " + v_id
		object_indiv_dfs[v_id].to_excel(write_file, sheet_name=indiv_sheet_name, index=False)